In [1]:
from pathlib import Path
from typing import Tuple

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.utils.data as data
import torchvision.transforms as T
from PIL import Image

from utils import locate_data

In [2]:
image_transforms = ["flip",
                    "crop",
                    "distorted_crop",
                    "jitter"]

In [3]:
data_path = Path("data/CamVid")

In [7]:
image_paths = sorted((data_path / "train").glob("*"))
mask_paths = sorted((data_path / "train_labels").glob("*"))

In [ ]:
for i in range(len(image_paths)):
    image_path = image_paths[i]
    mask_path = mask_paths[i]

    with open(image_path, "rb") as image_file, open(mask_path, "rb") as mask_file:
        image = Image.open(image_file)
        mask = Image.open(mask_file)

        image = torch.tensor(image)
        mask = torch.tensor(mask)

        